[トレーニングの概要 — センテンストランスフォーマーのドキュメント](https://gti.page.link/8FEwiMbAP3D6HtUm7)

[センテンストランスフォーマーのMNRトレーニング|データサイエンスに向けて](https://gti.page.link/GUzGHXmbctk3zyZ47)

[質問応答のために文-BERTを微調整する方法|キャピタルワン](https://gti.page.link/Thjje88sHJh8AREe9)

[Sentence BERTをFine TuningしてFAQを類似文書検索してみる - Taste of Tech Topics](https://gti.page.link/By2G5nNdJVqYiLF19)

[文トランスフォーマーモデルのトレーニングと微調整](https://gti.page.link/NwXJfCwYsNwiWPVc9)

[HuggingFace ブログ : Sentence Transformers モデルの訓練と微調整 – PyTorch 2.0, Hugging Face & Stable Diffusion](https://gti.page.link/85Za1AyVyyno6rXG6)

[Training and Fine-Tuning Sentence Transformers Models - Notebook Companion.ipynb - Colaboratory](https://gti.page.link/hHYGr8aD7DsxEJuj9)

In [1]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, models
from datasets import load_dataset
import pandas as pd
from torch.utils.data import DataLoader

from sentence_transformers import util
import torch

In [3]:
# モデルを読み込み
model_name = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2'
model_name = 'distilroberta-base'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# データセットを読み込み
dataset_id = "embedding-data/QQP_triplets"
# dataset_id = "embedding-data/sentence-compression"
dataset = load_dataset(dataset_id)

train_examples = []
train_data = dataset['train']['set']
n_examples = dataset['train'].num_rows // 10

for i in range(n_examples):
  example = train_data[i]
  train_examples.append([example['query'], example['pos'][0], example['neg'][0]])

datasets_df = pd.DataFrame(train_examples,columns=['anchor','positive','negative'])

Found cached dataset json (C:/Users/tmina/.cache/huggingface/datasets/embedding-data___json/embedding-data--QQP_triplets-1f161ec5c28ee86f/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# データローダーを構築
train_dataset = SentencesDataset([InputExample(texts=[row["anchor"], row["positive"], row["negative"]]) for index,row in datasets_df.iterrows()], model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

# ロス関数を設定
train_loss = losses.TripletLoss(model=model)

num_epochs = 2
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

# モデルの学習
model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=num_epochs,
        evaluation_steps=1,
        warmup_steps=warmup_steps,
        output_path="./sbert",
        )


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/636 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 12>:12                                                                            │
│                                                                                                  │
│    9 warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data             │
│   10                                                                                             │
│   11 # モデルの学習                                                                              │
│ ❱ 12 model.fit(                                                                                  │
│   13 │   │   train_objectives=[(train_dataloader, train_loss)],                                  │
│   14 │   │   epochs=num_epochs,                                                                  │
│   15 │   │   evaluation_steps=1,                                                                 │
│                                                                                                  │
│ c:\Users\tmina\anaconda3\lib\site-packages\sentence_transformers\SentenceTransformer.py:722 in   │
│ fit                                                                                              │
│                                                                                                  │
│   719 │   │   │   │   │   │   skip_scheduler = scaler.get_scale() != scale_before_step           │
│   720 │   │   │   │   │   else:                                                                  │
│   721 │   │   │   │   │   │   loss_value = loss_model(features, labels)                          │
│ ❱ 722 │   │   │   │   │   │   loss_value.backward()                                              │
│   723 │   │   │   │   │   │   torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad   │
│   724 │   │   │   │   │   │   optimizer.step()                                                   │
│   725                                                                                            │
│                                                                                                  │
│ c:\Users\tmina\anaconda3\lib\site-packages\torch\_tensor.py:488 in backward                      │
│                                                                                                  │
│    485 │   │   │   │   create_graph=create_graph,                                                │
│    486 │   │   │   │   inputs=inputs,                                                            │
│    487 │   │   │   )                                                                             │
│ ❱  488 │   │   torch.autograd.backward(                                                          │
│    489 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    490 │   │   )                                                                                 │
│    491                                                                                           │
│                                                                                                  │
│ c:\Users\tmina\anaconda3\lib\site-packages\torch\autograd\__init__.py:197 in backward            │
│                                                                                                  │
│   194 │   # The reason we repeat same the comment below is that                                  │
│   195 │   # some Python versions print out the first line of a multi-line function               │
│   196 │   # calls in the traceback and some print out the last line                              │
│ ❱ 197 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   198 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   199 │   │   allow_unreachable=True, accumulate_grad=True)  # Ca

In [ ]:
# 回答文書の一覧を取得
answers = list(datasets_df["positive"][~datasets_df["positive"].duplicated()])
# ベクトルに変換
corpus_embeddings = model.encode(answers, convert_to_tensor=True)

In [ ]:
# 質問文からの回答検索
query = "生徒登録がしたい"

query_embedding = model.encode(query, convert_to_tensor=True)
cos_scores = util.cos_sim(query_embedding , corpus_embeddings)
top_results =  torch.topk(cos_scores, k=3)

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 3 most similar sentences in corpus:")

for score, idx in zip(top_results[0][0], top_results[1][0]):
    print(answers[idx], "(Score: {:.4f})\n".format(score))


###  Sentence Transformerのfine tuning

In [6]:
modelB = SentenceTransformer('embedding-data/distilroberta-base-sentence-transformer')

In [7]:
dataset_id = "embedding-data/sentence-compression"
datasetB = load_dataset(dataset_id)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/tmina/.cache/huggingface/datasets/embedding-data___json/embedding-data--sentence-compression-d643585deb6e0073/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
train_examplesB = []
train_dataB = datasetB['train']['set']
n_examples = datasetB['train'].num_rows // 10

for i in range(n_examples):
  example = train_dataB[i]
  train_examplesB.append(InputExample(texts=[example[0], example[1]]))

In [12]:
train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=16)
train_lossB = losses.MultipleNegativesRankingLoss(model=modelB)
num_epochsB = 2
warmup_stepsB = int(len(train_dataloaderB) * num_epochsB * 0.1) #10% of train data

In [13]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1125 [00:00<?, ?it/s]

In [14]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

query_embedding = model.encode('How big is London')
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))


Similarity: tensor([[0.5472, 0.6330]])


In [15]:
from sentence_transformers import SentenceTransformer, util

question = "<Q>How many models can I host on HuggingFace?"
answer_1 = "<A>All plans come with unlimited private models and datasets."
answer_2 = "<A>AutoNLP is an automatic way to train and deploy state-of-the-art NLP models, seamlessly integrated with the Hugging Face ecosystem."
answer_3 = "<A>Based on how much training data and model variants are created, we send you a compute cost and payment link - as low as $10 per job."

model = SentenceTransformer('clips/mfaq')
query_embedding = model.encode(question)
corpus_embeddings = model.encode([answer_1, answer_2, answer_3])

print(util.semantic_search(query_embedding, corpus_embeddings))


c:\Users\tmina\anaconda3\lib\site-packages\transformers\configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


[[{'corpus_id': 0, 'score': 0.5646326541900635}, {'corpus_id': 2, 'score': 0.514234185218811}, {'corpus_id': 1, 'score': 0.4730040729045868}]]


#### Semantic Textual Similarity

In [29]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# Single list of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

#Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

print(len(sentences))
print(len(pairs))

for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))


8
8
28
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939
The cat sits outside 		 The cat plays in the garden 		 Score: 0.6788
I love pasta 		 Do you like pizza? 		 Score: 0.5096
I love pasta 		 The new movie is so great 		 Score: 0.2560
I love pasta 		 The new movie is awesome 		 Score: 0.2440
A man is playing guitar 		 The cat plays in the garden 		 Score: 0.2105
The new movie is awesome 		 Do you like pizza? 		 Score: 0.1969
The new movie is so great 		 Do you like pizza? 		 Score: 0.1692
The cat sits outside 		 A woman watches TV 		 Score: 0.1310
The cat plays in the garden 		 Do you like pizza? 		 Score: 0.0900


#### Paraphrase Mining

In [27]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# Single list of sentences - Possible tens of thousands of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

paraphrases = util.paraphrase_mining(model, sentences)

print(len(sentences))
print(len(paraphrases))

for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

    

8
28
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939
The cat sits outside 		 The cat plays in the garden 		 Score: 0.6788
I love pasta 		 Do you like pizza? 		 Score: 0.5096
I love pasta 		 The new movie is so great 		 Score: 0.2560
I love pasta 		 The new movie is awesome 		 Score: 0.2440
A man is playing guitar 		 The cat plays in the garden 		 Score: 0.2105
The new movie is awesome 		 Do you like pizza? 		 Score: 0.1969
The new movie is so great 		 Do you like pizza? 		 Score: 0.1692
The cat sits outside 		 A woman watches TV 		 Score: 0.1310
The cat plays in the garden 		 Do you like pizza? 		 Score: 0.0900


In [102]:
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer, util
import torch

# embedder = SentenceTransformer('all-MiniLM-L6-v2')
# embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
embedder = SentenceTransformer("sonoisa/sentence-bert-base-ja-mean-tokens-v2")
# embedder = SentenceTransformer("sonoisa/sentence-luke-japanese-base-lite")


# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'A cheetah is running behind its prey.',
          '日本語を混ぜてみる',
          ]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.',
          '日本語を混ぜてみる',]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    # top_results = torch.topk(cos_scores, k=top_k, largest=False)
    top_results = torch.topk(cos_scores, k=top_k, largest=True)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """





Query: A man is eating pasta.

Top 5 most similar sentences in corpus:
A man is eating a piece of bread. (Score: 0.6689)
A man is eating food. (Score: 0.6059)
A man is riding a horse. (Score: 0.5421)
A cheetah is running behind its prey. (Score: 0.5177)
A man is riding a white horse on an enclosed ground. (Score: 0.5014)




Query: Someone in a gorilla costume is playing a set of drums.

Top 5 most similar sentences in corpus:
A monkey is playing drums. (Score: 0.6640)
A man is riding a white horse on an enclosed ground. (Score: 0.5517)
A woman is playing violin. (Score: 0.5376)
A man is riding a horse. (Score: 0.4925)
Two men pushed carts through the woods. (Score: 0.3999)




Query: A cheetah chases prey on across a field.

Top 5 most similar sentences in corpus:
A cheetah is running behind its prey. (Score: 0.6613)
A man is riding a white horse on an enclosed ground. (Score: 0.4500)
A man is riding a horse. (Score: 0.4135)
A man is eating a piece of bread. (Score: 0.3873)
Two me

In [103]:
from scipy import spatial
import numpy as np

distances = 1-spatial.distance.cdist(corpus_embeddings, embedder.encode(queries, convert_to_tensor=True), 'cosine')
results = np.argmax(distances, axis=0)
# results = sorted(results, key=lambda x: x[1])

for idx, score in enumerate(results):
    print(queries[idx], corpus[score], "(Score: {:.4f})".format(distances[score][idx]))

A man is eating pasta. A man is eating a piece of bread. (Score: 0.6689)
Someone in a gorilla costume is playing a set of drums. A monkey is playing drums. (Score: 0.6640)
A cheetah chases prey on across a field. A cheetah is running behind its prey. (Score: 0.6613)
日本語を混ぜてみる 日本語を混ぜてみる (Score: 1.0000)
